In [2]:
# Preporocessing

In [1]:
import re
from transformers import pipeline
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Step 1: Parse SRT
def parse_srt(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    pattern = re.compile(r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\s(.*?)\n\n', re.DOTALL)
    matches = pattern.findall(content)
    timestamp_ranges = [(start, end, caption.replace('\n', ' ')) for start, end, caption in matches]
    return timestamp_ranges

# Step 2: Summarize Captions
def summarize_captions(timestamp_ranges, summarizer):
    summarized_ranges = []
    for start, end, caption in timestamp_ranges:
        summary = summarizer(caption, max_length=50, min_length=25, do_sample=False)[0]['summary_text']
        summarized_ranges.append((start, end, summary))
    return summarized_ranges

# Step 3: Extract Video Snippets
def convert_to_seconds(timestamp):
    hours, minutes, seconds = map(float, re.split('[:,]', timestamp))
    return hours * 3600 + minutes * 60 + seconds

def extract_video_snippets(video_path, summarized_ranges, output_dir):
    for i, (start, end, summary) in enumerate(summarized_ranges):
        start_seconds = convert_to_seconds(start)
        end_seconds = convert_to_seconds(end)
        output_file = f"{output_dir}/snippet_{i+1}.mp4"
        ffmpeg_extract_subclip(video_path, start_seconds, end_seconds, targetname=output_file)

# Example usage
srt_file = 'path_to_your_file.srt'
video_path = 'path_to_your_video.mp4'
output_dir = 'path_to_output_directory'

# Parse and summarize captions
timestamp_ranges = parse_srt(srt_file)
summarizer = pipeline('summarization')
summarized_ranges = summarize_captions(timestamp_ranges, summarizer)

# Extract video snippets
extract_video_snippets(video_path, summarized_ranges, output_dir)


/opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'moviepy'

In [2]:
pip install moviepy

  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
ERROR: Cannot uninstall decorator 5.1.1, RECORD file not found. Hint: The package was installed by brew.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.2.3/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
